In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdRGroupDecomposition as rdRGD
from rdkit.Geometry.rdGeometry import Point3D
# 显示图片
import Phe2BCH

import py3Dmol
def drawit(m,p=None,confId=-1):
        mb = Chem.MolToMolBlock(m,confId=confId)
        if p is None:
            p = py3Dmol.view(width=400,height=400)
        p.removeAllModels()
        p.addModel(mb,'sdf')
        p.setStyle({'stick':{}})
        p.setBackgroundColor('0xeeeeee')
        p.zoomTo()
        return p.show()

def mol_with_atom_index(mol):
    atoms = mol.GetNumAtoms()
    for idx in range(atoms):
        mol.GetAtomWithIdx(idx).SetProp('molAtomMapNumber',str(idx))
    return mol

In [30]:
ligands_smi = {}
with open("/home/chengyj/kinase_work/dataset/Bridged_ring/PDB_rings/PHE2BCH_pairs/lig_menu/AaaaA_only3.csv") as AaaaA:
    for line in AaaaA:
        info = line.split()
        ligands_smi[info[1]] = [info[0]]
inp = "YLJ_7m03"
key = inp.split("_")[0]
pdb_id=inp.split("_")[1]
lig_smi=ligands_smi[key][0]
# print(lig_smi)
pdb_path = f"/home/chengyj/kinase_work/dataset/Bridged_ring/PDB_rings/PHE2BCH_pairs/pdb_dataset/pdb/pdb{pdb_id}.ent"


# lig_Block = Phe2BCH.exctract_ligand_from_pdb(pdb_path,key,lig_smi,f"{key}_{pdb_id}_sin.pdb")



# test = Phe2BCH.phe2bch_topdb(lig_smi,lig_Block,f"{key}_{pdb_id}_bch.pdb")
# drawit(test)


In [64]:
lig_inp1 = Chem.MolFromSmiles("[H]N=C(N([H])C([H])([H])c1c([H])c([H])c([H])c(C([H])([H])N([H])[H])c1[H])C([H])([H])[H]")
lig_inp = Chem.MolFromSmiles(lig_smi)
lig_inp = Chem.AddHs(lig_inp)
AllChem.EmbedMolecule(lig_inp)
drawit(lig_inp)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [60]:
def exctract_ligand(pdb_path, lig_id, smiles):
    # 抽取pdbfile中的ligand部分,获得rdkit可读字符串形式的pdbBlock
    # 并加氢重命名后以pdb格式储存在outfile中
    mol_from_smi = Chem.MolFromSmiles(smiles)
    num = mol_from_smi.GetNumAtoms()
    # print(num)
    ligand_lines = []
    with open(pdb_path) as ent:
        for line in ent:
            if line[0:6] == "HETATM" and line[17:20] == lig_id:
                ligand_lines.append(line[0:78] + "  \n")
    # print("".join(ligand_lines))
    # print(ligand_lines)
    ligand_start = 0
    ligand_end = len(ligand_lines)
    for i in range(1, len(ligand_lines)):
        if ligand_lines[i][17:26] != ligand_lines[i - 1][17:26]:
            # print(ligand_lines[i][17:26])
            ligand_end = i
            if ligand_end - ligand_start >= num:
                break
            else:
                ligand_start = i
                ligand_end = len(ligand_lines)
        else:
            pass
    # print(ligand_end)
    # print(ligand_start)

    if ligand_end - ligand_start <= 2 * num:
        # avoid the altLoc is not A
        altLoc = ligand_lines[ligand_start][16]
        print(altLoc)
        # print(f"altLoc {altLoc}")
        if altLoc == "A" or altLoc == " " or altLoc == "1":
            lig_Block = "".join(ligand_lines[ligand_start:ligand_end])
        else:
            lig_Block = ""
            print("test")
            for line in ligand_lines[ligand_start:ligand_end]:
                print(line[16])
                if line[16] == altLoc:
                    lig_Block += line
            print(lig_Block)

    else:
        lig_Block = "".join(ligand_lines[ligand_start : ligand_start + num])
    print(lig_Block)
    lig_pdb = Chem.MolFromPDBBlock(lig_Block,flavor=1)
    return lig_pdb

In [61]:
lig_Block = exctract_ligand(pdb_path,key,lig_smi)
drawit(lig_Block)

A
HETATM 4540  C11AYLJ B 401     -17.212  -5.878   4.966  0.50 46.10           C  
HETATM 4541  C16AYLJ B 401     -15.083  -3.396   2.051  0.50 58.73           C  
HETATM 4542  C17AYLJ B 401     -16.421  -4.085   2.441  0.50 53.30           C  
HETATM 4543  O18AYLJ B 401     -16.367  -1.199   4.466  0.50 49.20           O  
HETATM 4544  C23AYLJ B 401     -22.316  -2.749   0.139  0.50 56.35           C  
HETATM 4545  C02AYLJ B 401     -20.804  -3.856   1.845  0.50 53.51           C  
HETATM 4546  C04AYLJ B 401     -20.990  -5.768   3.515  0.50 43.77           C  
HETATM 4547  C05AYLJ B 401     -21.685  -7.124   3.841  0.50 43.13           C  
HETATM 4548  C06AYLJ B 401     -23.226  -7.087   4.033  0.50 47.16           C  
HETATM 4549  C07AYLJ B 401     -23.843  -8.435   4.442  0.50 50.19           C  
HETATM 4550  C08AYLJ B 401     -23.621  -6.003   5.040  0.50 51.67           C  
HETATM 4551  C09AYLJ B 401     -19.452  -5.990   3.712  0.50 46.26           C  
HETATM 4552  C12AYLJ B 401

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [62]:
def getconf_from_missingatom_pdb(refmol,inpmol):
    inpmol = Chem.AddHs(inpmol)
    AllChem.EmbedMolecule(inpmol, maxAttempts=50000)
    inpmol = Chem.RemoveAllHs(inpmol)

    mcs = rdFMCS.FindMCS([refmol, inpmol],timeout=3, completeRingsOnly=True,bondCompare=rdFMCS.BondCompare.CompareAny)
    bonded_conf = refmol.GetConformer()
    conf_res = inpmol.GetConformer()
    inpmol_prop = Chem.rdForceFieldHelpers.MMFFGetMoleculeProperties(inpmol)
    if inpmol_prop:
        ff_mcs = Chem.rdForceFieldHelpers.MMFFGetMoleculeForceField(inpmol, inpmol_prop)
    else:
        ff_mcs = Chem.rdForceFieldHelpers.UFFGetMoleculeForceField(inpmol)

    for i, j in zip(refmol.GetSubstructMatch(mcs.queryMol), inpmol.GetSubstructMatch(mcs.queryMol)):
        ff_mcs.AddFixedPoint(j)
        conf_res.SetAtomPosition(j, bonded_conf.GetAtomPosition(i))
    for i in range(10):
        try:
            ff_mcs.Minimize()
            pass
        except:
            pass
    Chem.MolToPDBBlock(inpmol)
    return inpmol

In [65]:
drawit(getconf_from_missingatom_pdb(lig_Block,lig_inp))

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [17]:
ligands_smi = {}
with open(
        "/home/chengyj/kinase_work/dataset/Bridged_ring/PDB_rings/PHE2BCH_pairs/lig_menu/AaaaA_only4.csv"
    ) as AaaaA:
        for line in AaaaA:
            info = line.split()
            ligands_smi[info[1]] = [info[0],False]
    # print(1)
with open(
        "/home/chengyj/kinase_work/dataset/Bridged_ring/PDB_rings/PHE2BCH_pairs/pdb_dataset/cc-to-pdb.tdd"
    ) as ligs:
        for line in ligs:
            info = line.split()
            if info[0] in ligands_smi.keys():
                ligands_smi[info[0]].append(info[1:])
                ligands_smi[info[0]][1] = True
    # print(2)
count = 0
for key in ligands_smi:
    count += 1
    if ligands_smi[key][1]:
        for pdbid in ligands_smi[key][2]:
            print(f"{count} {key}_{pdbid}")

1 009_3ckr
2 012_3ckp
4 038_3kyr
5 03Q_3pp0
6 043_3saz
7 059_2pj6
8 069_2wa4
9 06Z_3rjc
10 07Q_3tv7
11 07R_3twj
12 0FS_3vc4
14 0GO_3veu
15 0J7_4dds
16 0K9_4di2
17 0KF_3vo3
18 0N3_4e3i
19 0OO_4eh7
20 0OP_4eh8
21 0RY_4f1l
22 0RZ_4f1q
23 0UQ_4fm8
24 0V7_4fph
25 0V8_4fpy
26 0YZ_4h3c
27 11B_2pj5
28 11R_2cg1
29 14D_4hej
30 14W_1foi
30 14W_1qwc
30 14W_1qw5
31 15V_4hge
32 160_1nmq
33 16F_3mrv
34 16K_4ek8
35 16U_3dt0
36 16W_4hnf
36 16W_4hni
37 16X_2x4f
38 183_1u1f
39 18P_3u6a
40 19U_2zfp
41 1AW_3f3u
42 1D9_4ibj
43 1FB_4ihp
43 1FB_5f0a
44 1FO_4iqv
44 1FO_4iqt
45 1FV_4im0
45 1FV_6qau
45 1FV_4iwq
46 1H4_4iwo
47 1LC_4jg7
48 1N6_4jx7
49 1NH_1o1s
50 1QE_4ugm
50 1QE_4kci
51 1QU_4k9h
52 1RG_4qu3
52 1RG_4gcs
52 1RG_3zgp
52 1RG_7kgn
52 1RG_7kgm
52 1RG_3m6b
52 1RG_6skr
52 1RG_6p99
53 1RI_7f8g
54 1SF_4kpz
55 1SI_7f8h
56 1VJ_4oyb
57 1WS_4bkz
58 1ZC_4oym
59 21U_2zgb
60 21Z_3rni
61 22O_3mta
62 22U_2zc9
62 22U_5mls
63 22X_3km4
64 235_2pj7
65 23D_2f57
65 23D_2izt
65 23D_6guf
65 23D_2cdz
66 23U_3dhk
66 23U_5mjt
